# TEMAS - Turkey Earthquake Monitoring and Analysis System

### ==== Version: 0.7.1 released on 12 March 2023 by Marcus Zou ====


## Data Source

- The origin data table is embedded/nested at http://www.koeri.boun.edu.tr/sismo/2/latest-earthquakes/automatic-solutions/, 
- being updated daily per events happened in Turkey and surounding areas.

- From 16 Jan 2023 and onwards, I started to grab down the dataset daily and manually, 
- and put/merge into a csv file named "data/koeri.boun.edu.tr-lasteq-autosol_from_2023-01-16_tab.csv". 
- This is a very time consuming.


## Progress

#### v0.7.1 build 2023-03-12
* Dockerized the project into a cloud service
* Scheduled a Data Updater

#### v0.7.0 build 2023-03-11
* Failed to add Choropleth map due to lack of decent geojson file per province.
* Re-org the project files and folders prior to deployment.
  
#### v0.6.0 build 2023-03-10
* Split the jobs (Daily Job Runner + Mapper)
* Designed and Made a landing page (index.html) and other pages.

#### v0.5.0 build 2023-03-09
* Organized the all-in-one Jupyter Notebook: db-reader + scrapper + merger + mapper.

#### v0.4.1 build 2023-03-08
* Scraping the multiple pages from koeri.boun.edu.tr was successful.
* http://sc3.koeri.boun.edu.tr/eqevents/events1.html
* http://sc3.koeri.boun.edu.tr/eqevents/events2.html
* ...
* http://sc3.koeri.boun.edu.tr/eqevents/events32.html
* http://sc3.koeri.boun.edu.tr/eqevents/events33.html

#### v0.4.0 build 2023-03-07
* Scraping the first page from koeri.boun.edu.tr was successful.
* http://sc3.koeri.boun.edu.tr/eqevents/events.html

#### v0.3.0 build 2023-03-04
* Created a SQLite3 db and saved the dataframe of Historic data into it.

#### v0.2.1 build 2023-02-27
* Merged historic and real-time data into one local dataframe.

#### v0.2.0 build 2023-02-26
* Historic database added (from 16 Jan 2023).

#### v0.1.0 build 2023-02-13
* First release - current 500 datapoints only.


## Planning

Thinking about -
* to load up the historic data - Done!
* to build a local database (SQLite3?) to save what we have grabbed down so far - Done!
* to run a webScraping task to grab down the daily updates from the website mentioned above - Done!
* to merge the historic dataset and the realtime daily updates into one database - Done!
<br><br>
* Finnally implement this into .py files and schedule daily job-runner - Done!
    

## PART 0. Prep

In [1]:
# Load up libraries
import os
import pandas as pd
import folium
from folium import plugins
import branca
import geocoder
import reverse_geocoder as rg
import webbrowser
import datetime as dt
import sqlite3
import requests
from bs4 import BeautifulSoup

print('Pandas: ', pd.__version__)
print('Folium: ', folium.__version__)

Pandas:  1.5.3
Folium:  0.14.0


## PART 1. Web Scrapping Real-time Data


More in-depth digging reveals that the actual tables are in HTML format as below (34 counts):

* http://sc3.koeri.boun.edu.tr/eqevents/events.html
* http://sc3.koeri.boun.edu.tr/eqevents/events1.html
* http://sc3.koeri.boun.edu.tr/eqevents/events2.html
* ...
* http://sc3.koeri.boun.edu.tr/eqevents/events32.html
* http://sc3.koeri.boun.edu.tr/eqevents/events33.html

Then the target page is being reset to: http://sc3.koeri.boun.edu.tr/eqevents/events{i}.html and 
the scrapping task focus on the first a few pages if the job runner be turned on every day.

It turns out the first page naming convention **differs** from the rest, then we have to scrap them into seperate dataframes and merge the two.

Then merge the dataframe to the SQLite3 database.


In [2]:
# send a GET request to the URL - the first page
url0 = "http://sc3.koeri.boun.edu.tr/eqevents/events.html"
response = requests.get(url0)
# parse the HTML content of the page with BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

data0 = []
# find the table element and iterate over its rows
table = soup.find("table", {'class': 'index'})
rows = table.find_all("tr", {'class': 'trIndevnrow'})[1:]
for row in rows:
    # get the cells in the row
    cols = row.find_all("td")
    # get the earthquake information from the columns
    origintimeutc = cols[0].text.strip()
    magnitude = cols[1].text.strip()
    magType = cols[2].text.strip()
    lat = cols[3].text.strip()
    long = cols[4].text.strip()
    depth = cols[5].text.strip()
    region = cols[6].text.strip()
    am = cols[7].text.strip()
    lastUpd = cols[8].text.strip()
    kml = cols[9].text.strip()
    
    data0.append([origintimeutc, magnitude, magType, lat, long, depth, region, am, lastUpd, kml])
        
df0 = pd.DataFrame(data0, columns=['origintimeutc', 'magnitude', 'magType', 'latitude', 'longitude', 'depthKM', 'region', 'measMethod', 'updTime', 'attribute'])
df0

,origintimeutc,magnitude,magType,latitude,longitude,depthKM,region,measMethod,updTime,attribute
0,2023/03/12 16:52:33,3.5,MLv,38.1839° N,37.0761° E,0,Elbistan-Kahramanmaras,A,2023/03/12 16:56:24,Stations Event
1,2023/03/12 16:34:45,3.3,MLv,38.1409° N,36.9543° E,5,Afsin-Kahramanmaras,A,2023/03/12 16:37:25,Stations Event
2,2023/03/12 15:06:23,3.0,MLv,37.9541° N,36.3721° E,10,Goksun-Kahramanmaras,A,2023/03/12 15:08:47,Stations Event
3,2023/03/12 13:01:22,3.5,MLv,38.0667° N,36.5378° E,0,Goksun-Kahramanmaras,A,2023/03/12 13:04:58,Stations Event
4,2023/03/12 12:22:15,3.0,MLv,36.9690° N,28.0481° E,10,Gokova Korfezi-Ege Denizi,A,2023/03/12 12:25:56,Stations Event
5,2023/03/12 10:35:17,3.0,MLv,36.6815° N,36.4288° E,0,Hassa-Hatay,A,2023/03/12 10:39:06,Stations Event
6,2023/03/12 09:25:20,2.9,MLv,38.0618° N,37.9076° E,5,Dogansehir-Malatya,A,2023/03/12 09:28:52,Stations Event
7,2023/03/12 09:10:27,2.4,MLv,40.7573° N,27.3607° E,11,Marmara Denizi (Bati),A,2023/03/12 09:13:54,Stations Event
8,2023/03/12 01:42:57,2.6,MLv,39.6217° N,25.7152° E,10,Aegean Sea,A,2023/03/12 01:45:50,Stations Event
9,2023/03/12 00:04:12,2.7,MLv,37.8644° N,36.5973° E,11,Merkez-Kahramanmaras,A,2023/03/12 00:07:42,Stations Event


In [3]:
# Scrapping Multiple Pages
data1 = []
# loop through the pages 1-5
for i in range(1,9):
    # specify the URL to scrape
    url = f"http://sc3.koeri.boun.edu.tr/eqevents/events{i}.html"
    
    # make a GET request to the URL and get the HTML content
    response = requests.get(url)
    html_content = response.content
    
    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # find the earthquake table
    eq_table = soup.find('table', {'class': 'index'})
    
    # loop through the rows of the table
    for row in eq_table.find_all('tr', {'class': 'trIndevnrow'})[1:]:
        # get the columns of the row
        cols = row.find_all('td')
        
        # get the earthquake information from the columns
        origintimeutc = cols[0].text.strip()
        magnitude = cols[1].text.strip()
        magType = cols[2].text.strip()
        lat = cols[3].text.strip()
        long = cols[4].text.strip()
        depth = cols[5].text.strip()
        region = cols[6].text.strip()
        am = cols[7].text.strip()
        lastUpd = cols[8].text.strip()
        kml = cols[9].text.strip()
        
        data1.append([origintimeutc, magnitude, magType, lat, long, depth, region, am, lastUpd, kml])
               
df1 = pd.DataFrame(data1, columns = ["origintimeutc", "magnitude", "magType", "latitude", "longitude", "depthKM", "region", "measMethod", "updTime", "attribute"])
df1


,origintimeutc,magnitude,magType,latitude,longitude,depthKM,region,measMethod,updTime,attribute
0,2023/03/11 06:37:50,3.3,MLv,37.1869° N,36.1725° E,8,Merkez-Osmaniye,A,2023/03/11 06:41:04,Stations Event
1,2023/03/11 06:25:31,3.1,MLv,40.8334° N,31.7148° E,0,Merkez-Bolu,A,2023/03/11 06:28:22,Stations Event
2,2023/03/11 05:39:22,3.1,MLv,38.0832° N,37.2079° E,5,Ekinozu-Kahramanmaras,A,2023/03/11 05:41:56,Stations Event
3,2023/03/11 04:30:06,3.7,MLv,36.5682° N,27.1557° E,5,"Dodecanese Islands, Greece",A,2023/03/11 04:32:34,Stations Event
4,2023/03/11 03:02:44,1.8,MLv,40.7611° N,29.1852° E,0,Marmara Denizi (Dogu),A,2023/03/11 03:04:14,Stations Event
...,...,...,...,...,...,...,...,...,...,...
101,2023/03/04 22:55:23,3.9,MLv,44.9725° N,22.9999° E,0,Romania,A,2023/03/04 23:14:04,Stations Event
102,2023/03/04 21:40:08,2.3,MLv,37.3663° N,37.0844° E,6,Pazarcik-Kahramanmaras,A,2023/03/04 21:43:02,Stations Event
103,2023/03/04 20:59:41,4.0,MLv,38.4677° N,40.3299° E,0,Hani-Diyarbakir,A,2023/03/04 21:16:54,Stations Event
104,2023/03/04 20:48:41,2.4,MLv,37.4652° N,37.0331° E,9,Merkez-Kahramanmaras,A,2023/03/04 20:51:14,Stations Event


In [4]:
# Concatenate 2 realtime dataframes
rtDF = pd.concat([df0, df1]).drop_duplicates(subset='origintimeutc', keep='last')
# Reset the Index
rtDF = rtDF.reset_index(drop=True)
rtDF

,origintimeutc,magnitude,magType,latitude,longitude,depthKM,region,measMethod,updTime,attribute
0,2023/03/12 16:52:33,3.5,MLv,38.1839° N,37.0761° E,0,Elbistan-Kahramanmaras,A,2023/03/12 16:56:24,Stations Event
1,2023/03/12 16:34:45,3.3,MLv,38.1409° N,36.9543° E,5,Afsin-Kahramanmaras,A,2023/03/12 16:37:25,Stations Event
2,2023/03/12 15:06:23,3.0,MLv,37.9541° N,36.3721° E,10,Goksun-Kahramanmaras,A,2023/03/12 15:08:47,Stations Event
3,2023/03/12 13:01:22,3.5,MLv,38.0667° N,36.5378° E,0,Goksun-Kahramanmaras,A,2023/03/12 13:04:58,Stations Event
4,2023/03/12 12:22:15,3.0,MLv,36.9690° N,28.0481° E,10,Gokova Korfezi-Ege Denizi,A,2023/03/12 12:25:56,Stations Event
...,...,...,...,...,...,...,...,...,...,...
114,2023/03/04 22:55:23,3.9,MLv,44.9725° N,22.9999° E,0,Romania,A,2023/03/04 23:14:04,Stations Event
115,2023/03/04 21:40:08,2.3,MLv,37.3663° N,37.0844° E,6,Pazarcik-Kahramanmaras,A,2023/03/04 21:43:02,Stations Event
116,2023/03/04 20:59:41,4.0,MLv,38.4677° N,40.3299° E,0,Hani-Diyarbakir,A,2023/03/04 21:16:54,Stations Event
117,2023/03/04 20:48:41,2.4,MLv,37.4652° N,37.0331° E,9,Merkez-Kahramanmaras,A,2023/03/04 20:51:14,Stations Event


## PART 2. Load Historic Data from Local Database


In [5]:
# Create a connection to the databse
conn = sqlite3.connect('data/earthquake-in-turkey.db')

# Read out the whole dataset as dataframe
histDF = pd.read_sql_query("SELECT * FROM quaketk", conn)
histDF

,origintimeutc,magnitude,magtype,latitude,longitude,depthkm,region,measmethod,updtime,attribute
0,2023/01/16 21:35:00,2.5,MLv,39.5220° N,25.9825° E,7.0,Ayvacik Aciklari-EGE DENIZI,M,2023/01/17 09:38:00,Stations Event
1,2023/01/16 21:58:00,2.9,MLv,39.4010° N,26.2998° E,3.0,Turkey,M,2023/01/17 10:24:00,Stations Event
2,2023/01/16 22:05:00,3.4,MLv,36.9406° N,26.5903° E,7.0,"Dodecanese Islands, Greece",M,2023/01/17 10:41:00,Stations Event
3,2023/01/16 22:07:00,3.1,MLv,39.3431° N,26.2384° E,7.0,Midilli Adasi-Ege Denizi,M,2023/01/17 10:44:00,Stations Event
4,2023/01/16 22:19:00,2.4,MLv,39.5278° N,25.9722° E,0.0,Aegean Sea,M,2023/01/17 10:47:00,Stations Event
...,...,...,...,...,...,...,...,...,...,...
2047,2023/03/12 12:22:15,3.0,MLv,36.9690° N,28.0481° E,10.0,Gokova Korfezi-Ege Denizi,A,2023/03/12 12:25:56,Stations Event
2048,2023/03/12 13:01:22,3.5,MLv,38.0667° N,36.5378° E,0.0,Goksun-Kahramanmaras,A,2023/03/12 13:04:58,Stations Event
2049,2023/03/12 15:06:23,3.0,MLv,37.9541° N,36.3721° E,10.0,Goksun-Kahramanmaras,A,2023/03/12 15:08:47,Stations Event
2050,2023/03/12 16:34:45,3.3,MLv,38.1409° N,36.9543° E,5.0,Afsin-Kahramanmaras,A,2023/03/12 16:37:25,Stations Event


In [6]:
# Determine the last timestamp of the Historic database
cutoff_time = histDF['origintimeutc'].max()
print('The newest timestamp in Historic database is:', cutoff_time)

# Subset the Realtime DF per the last record in the Historic Database
rtDF = rtDF[rtDF['origintimeutc'] > cutoff_time]

# Sort the DF to be the format of: Older datapoint to be inserted firstly
rtDF1 = rtDF.sort_values(by='origintimeutc', ascending=True)
rtDF1.reset_index(drop=True, inplace=True)
rtDF1


The newest timestamp in Historic database is: 2023/03/12 16:52:33


,origintimeutc,magnitude,magType,latitude,longitude,depthKM,region,measMethod,updTime,attribute


## PART 3. Merge/Insert Realtime DF into Historic Database

In [7]:
# Save the dataframe to database
rtDF1.columns = ['origintimeutc', 'magnitude', 'magtype', 'latitude', 'longitude', 'depthkm', 'region', 'measmethod', 'updtime', 'attribute']
rtDF1.to_sql('quaketk', conn, if_exists='append', index=False)


0

In [8]:
# Verify if the dataframe has been saved to sqlite DB or not, please uncomment the following command
#conn.execute('SELECT * from quaketk ORDER BY origintimeutc DESC LIMIT 20').fetchall()

In [9]:
# Cloe the Database
conn.close

<function Connection.close()>

## PART 4. Retrieve the Whole Dataset for Export and Display

In [10]:
# Setup pandas display template
pd.set_option('display.width', 960)
pd.set_option('display.max_columns', 9)

In [11]:
# Create a connection to the databse
conn = sqlite3.connect('data/earthquake-in-turkey.db')

# Read out the whole dataset as dataframe
df_d = pd.read_sql_query("SELECT * FROM quaketk", conn)
df_d

,origintimeutc,magnitude,magtype,latitude,...,region,measmethod,updtime,attribute
0,2023/01/16 21:35:00,2.5,MLv,39.5220° N,...,Ayvacik Aciklari-EGE DENIZI,M,2023/01/17 09:38:00,Stations Event
1,2023/01/16 21:58:00,2.9,MLv,39.4010° N,...,Turkey,M,2023/01/17 10:24:00,Stations Event
2,2023/01/16 22:05:00,3.4,MLv,36.9406° N,...,"Dodecanese Islands, Greece",M,2023/01/17 10:41:00,Stations Event
3,2023/01/16 22:07:00,3.1,MLv,39.3431° N,...,Midilli Adasi-Ege Denizi,M,2023/01/17 10:44:00,Stations Event
4,2023/01/16 22:19:00,2.4,MLv,39.5278° N,...,Aegean Sea,M,2023/01/17 10:47:00,Stations Event
...,...,...,...,...,...,...,...,...,...
2047,2023/03/12 12:22:15,3.0,MLv,36.9690° N,...,Gokova Korfezi-Ege Denizi,A,2023/03/12 12:25:56,Stations Event
2048,2023/03/12 13:01:22,3.5,MLv,38.0667° N,...,Goksun-Kahramanmaras,A,2023/03/12 13:04:58,Stations Event
2049,2023/03/12 15:06:23,3.0,MLv,37.9541° N,...,Goksun-Kahramanmaras,A,2023/03/12 15:08:47,Stations Event
2050,2023/03/12 16:34:45,3.3,MLv,38.1409° N,...,Afsin-Kahramanmaras,A,2023/03/12 16:37:25,Stations Event


In [12]:
# Print the dataset range
print('Time Range of the Dataset is between:', df_d['origintimeutc'].min(), 'and', df_d['origintimeutc'].max())

print("\ndf.columns:\n", df_d.columns)

Time Range of the Dataset is between: 2023/01/16 21:35:00 and 2023/03/12 16:52:33

df.columns:
 Index(['origintimeutc', 'magnitude', 'magtype', 'latitude', 'longitude', 'depthkm', 'region', 'measmethod', 'updtime', 'attribute'], dtype='object')


## PART 5. Data Re-treatment and Export Data Table in HTML format

In [13]:
# Re-treatment of the dataset
df_d['origintimeutc'] = df_d['origintimeutc'].apply(lambda x: dt.datetime.strptime(x,'%Y/%m/%d %H:%M:%S') if type(x)==str else pd.NaT)
df_d['magnitude'] = df_d['magnitude'].astype('float')
df_d['magtype'] = df_d['magtype'].astype('string')

df_d['latitude'] = df_d['latitude'].astype(str).map(lambda x: x.rstrip('° N').rstrip('° S'))
df_d['longitude'] = df_d['longitude'].astype(str).map(lambda x: x.rstrip('° E').rstrip('° W'))
df_d['depthkm'] = df_d['depthkm'].replace('-', 0).astype("float")

df_d['region'] = df_d['region'].astype('string')
df_d['measmethod'] = df_d['measmethod'].astype('string')
df_d['updtime'] = df_d['updtime'].apply(lambda x: dt.datetime.strptime(x,'%Y/%m/%d %H:%M:%S') if type(x)==str else pd.NaT)
df_d['attribute'] = df_d['attribute'].astype('string')

# Adjust datetime from UTC (GMT) to Turkey timezone (GMT+3)
df_d['eventtime'] = df_d['origintimeutc'] + pd.DateOffset(hours=3)
df_d['updtime'] = df_d['updtime'] + pd.DateOffset(hours=3)
df_d['eventtime'] = pd.to_datetime(df_d['eventtime'])
df_d['updtime'] = pd.to_datetime(df_d['updtime'])


# Create new columns for date and time
df_d['date'] = pd.to_datetime(df_d['eventtime']).dt.date
df_d['time'] = pd.to_datetime(df_d['eventtime']).dt.time

# take a look
df_d.tail()


,origintimeutc,magnitude,magtype,latitude,...,attribute,eventtime,date,time
2047,2023-03-12 12:22:15,3.0,MLv,36.9690,...,Stations Event,2023-03-12 15:22:15,2023-03-12,15:22:15
2048,2023-03-12 13:01:22,3.5,MLv,38.0667,...,Stations Event,2023-03-12 16:01:22,2023-03-12,16:01:22
2049,2023-03-12 15:06:23,3.0,MLv,37.9541,...,Stations Event,2023-03-12 18:06:23,2023-03-12,18:06:23
2050,2023-03-12 16:34:45,3.3,MLv,38.1409,...,Stations Event,2023-03-12 19:34:45,2023-03-12,19:34:45
2051,2023-03-12 16:52:33,3.5,MLv,38.1839,...,Stations Event,2023-03-12 19:52:33,2023-03-12,19:52:33


In [14]:
# Interactive data table using Dtale - Big fun, isn't it?
import dtale
dtale.show(df_d)

In [15]:
# Preapre Data for exporting to a Data Table

# Select columns
df_tbl = df_d[['eventtime', 'magnitude', 'magtype', 'latitude', 'longitude', 'depthkm', 'region', 'measmethod', 'updtime', 'attribute']]
df_tbl.columns = ['eventTime', 'magnitude', 'magType', 'latitude', 'longitude', 'depthKm', 'region', 'measMethod', 'updTime', 'attribute']
# Change the default Ascending to Descending to put newest datapoints on the top of the data table
df_tbl_1 = df_tbl.sort_values(by='eventTime', ascending=False)

In [29]:
# Self-making Interactive Data Table using JQuery
def generate_html(dataframe: pd.DataFrame):
    # get the table HTML from the dataframe
    table_html = dataframe.to_html(table_id="table")
    # construct the complete HTML with jQuery Data tables
    # You can disable paging or enable y scrolling on lines 20 and 21 respectively
    html = f"""
    <html>
    <header>
        <link href="https://cdn.datatables.net/1.11.5/css/jquery.dataTables.min.css" rel="stylesheet">
    </header>
    <body>
    {table_html}
    <script src="https://code.jquery.com/jquery-3.6.0.slim.min.js" integrity="sha256-u7e5khyithlIdTpu22PHhENmPcRdFiHRjhAuHcs05RI=" crossorigin="anonymous"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.11.5/js/jquery.dataTables.min.js"></script>
    <script>
        $(document).ready( function () {{
            $('#table').DataTable({{
                order: [[0, 'desc']],
                lengthMenu: [20, 50, 100, 200],
                pageLength: 20,
                paging: true,
            }});
        }});
    </script>
    </body>
    </html>
    """
    # return the html
    return html

# Give a go
table_interact = generate_html(df_tbl_1)
with open("output/table_interact_koeri.html", 'w') as f:
    f.write(table_interact)
webbrowser.open(os.getcwd() + "../output/table_interact_koeri.html")


True

In [ ]:
# Another Pretty HTML Data Table
from pretty_html_table import build_table

html_table_bluelight = build_table(df_tbl_1, 'blue_light',
                                   font_size='medium',
                                   font_family='Arial',
                                   text_align='center',
                                   width='auto',
                                   index=False,
                                   conditions={
                                        'magnitude': {
                                        'min': 3.0,
                                        'max': 9.0,
                                        'mix_color': 'green',
                                        'max_color': 'red'
                                        }
                                   },
                                   even_color='black',
                                   even_bg_color='white'
                                   )

# center the table using HTML formatting
html_table_bluelight = '<div style="text-align:center">' + html_table_bluelight + '</div>'

# write the HTML table to a file
with open('output/table_bluelight_koeri.html', 'w') as f:
    f.write(html_table_bluelight)
webbrowser.open(os.getcwd() + "../output/table_bluelight_koeri.html")


## PART 6. Export Maps

### 6.1 Bubble Map

In [ ]:
# Subset the df and rename the columns
df_d2 = df_d[['date', 'time', 'latitude', 'longitude', 'depthkm', 'magnitude', 'magtype', 'region', 'measmethod', 'eventtime', 'updtime', 'attribute']]
df_d2.head()
# print the earthquake with magnitude >=4
print(df_d2[df_d2['magnitude'].astype('float') >= 5])

In [ ]:
# Define our functions
# Func 1 - Proportionize the fill_color against magnitude
myColorScheme = ['green', 'red', 'black']
def get_color(magnitude):
    if magnitude <= 3:
        return myColorScheme[0]
    elif magnitude <= 6:
        return myColorScheme[1]
    else:
        return myColorScheme[2]

# Func 2 - addCircles
popwidth, popht = 250, 150
def addCircles(df, map):
    for x, y, mag, region, date, time in zip(df['latitude'], df['longitude'], df['magnitude'], df['region'], df['date'], df['time']):
        popUp = f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Magnitude: {mag} </strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Region: {region}</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Date: {date}</strong></span></p>" \
                f"<p style='text-align: center;'><span style='font-family: Verdana, Geneva, sans-serif; font-size: 12px; color: rgb(40, 50, 78);'><strong>Time: {time}</strong></span></p>"
        iframe = folium.IFrame(popUp, width=popwidth, height=popht)
        popup = folium.Popup(iframe, max_width=450)
        folium.CircleMarker(location=(x, y), radius=float(mag) * 4, weight=2, opacity=1, popup=popup,
                            color=get_color(mag), fill_color=get_color(mag), fill_opacity=0.6).add_to(map)

In [ ]:
# Map Parameters
magThreshold = 4
mapCtr = [39.16, 35.66]

# Initial Map
bubbleMap = folium.Map(location=mapCtr, zoom_start=6, tiles=None)

# Load up the tectonic polygon (in GeoJson format)
# Original URL - https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json
folium.GeoJson('data/PB2002_boundaries.json', name="Tectonic Boundaries").add_to(bubbleMap)

# Add tiles with custom names (The first tile will be the default)
folium.TileLayer('Stamen Terrain', name='Terrian').add_to(bubbleMap)
folium.TileLayer('openstreetmap', name='Open Street').add_to(bubbleMap)
folium.TileLayer('Stamen Toner', name='Toner').add_to(bubbleMap)
folium.TileLayer('Stamen Water Color', name='Water Color').add_to(bubbleMap)
folium.TileLayer('cartodbdark_matter', name='Dark Matter').add_to(bubbleMap)
folium.LayerControl().add_to(bubbleMap)

# Map Data filtering
earthquakeDF = df_d2[df_d2['magnitude'].astype('float') >= magThreshold]

# add Circles (sizing per magnitude)
addCircles(earthquakeDF, bubbleMap)

# Add lenend to the bubble map
from branca.colormap import StepColormap
magMin, magLow, magHigh, magMax = 0, 3, 6, 10
mySteps = [magMin, magLow, magHigh, magMax]

# myColorScheme has been defined previously
legend_bar = StepColormap(colors=myColorScheme,
                          index=mySteps,
                          vmin=magMin, vmax=magMax,
                          tick_labels = mySteps)
legend_bar.caption = 'Magnitude Scale'
legend_bar.add_to(bubbleMap)

# More widgets
bubbleMap.add_child(folium.LatLngPopup())
bubbleMap

In [ ]:
# Eventually save map to interactive HTML file and display in the browser
bubbleMap.save("output/map_bubble_koeri.html")
webbrowser.open(os.getcwd() + "../output/map_bubble_koeri.html")

### 6.2 Heat Map

In [ ]:
# Heat Map
import ipywidgets as widgets
from IPython.display import display

# Make a list of heatMapData
heatMapData = earthquakeDF[['latitude', 'longitude', 'magnitude']]

# Create the HeatMap - "CartoDB Dark_Matter" is the default tile for heat map
heatMap = folium.Map(mapCtr, tiles="Stamen Terrain", zoom_start=6)
# More widgets
heatMap.add_child(folium.LatLngPopup())

# Create heat map
plugins.HeatMap(heatMapData, 
                name="Magnitude", 
                radius=earthquakeDF['magnitude'].mean()*3,
                min_opacity = 1,
                max_zoom=15,
                blur=20,
                overlay=False,
                control=False,
                show=True
                ).add_to(heatMap)

# Create a toolbar using ipywidgets in Jupyter Notebook
toolbar_left = widgets.HTML(
    value='<h3 style="text-align:left; background-color: purple; color: white">Turkey Earthquake Monitoring System - Heat Map</h3>',
    layout=widgets.Layout(width='70%', height='50px')
)
toolbar_right = widgets.HTML(
    value='<h3 style="text-align:right; background-color: light-grey; color: blue"> \
    <a href="output/map_bubble_koeri.html" target="_blank">Switch to Bubble Map</a></h3>',
    layout=widgets.Layout(width='30%', height='50px')
)

toolbar = widgets.HBox([toolbar_left, toolbar_right])

# Display the toolbar and the map together using IPython.display
display(toolbar)
display(heatMap)


In [ ]:
# Save Heatmap and display in the browser
heatMap.save("output/map_heat_koeri.html")
webbrowser.open(os.getcwd() + "../output/map_heat_koeri.html")

### 6.3 Choropleth Map

__Per Wikipedia__

A choropleth map (from Greek χῶρος (choros) 'area/region', and πλῆθος (plethos) 'multitude') is a type of statistical thematic map that uses pseudocolor, i.e., color corresponding with an aggregate summary of a geographic characteristic within spatial enumeration units, such as population density or per-capita income.

```
#####  The data sample have issue, then disable this for now
# Map Parameters
magThreshold = 4
mapCtr = [39.16, 35.66]

# Load up the turkey provinces polygon (in GeoJson format)
province_geo = pd.read_json('data/geoboundaries-TUR-ADM1_simplified.json')
province_data = pd.read_csv("data/geodata-TUR-ADM1_mag_sum_avg.csv")
#province_data.columns

merged_data = province_geo.merge(province_data, left_on='shapeISO', right_on='shapeISO')

# Initial Map
choropMap = folium.Map(location=mapCtr, zoom_start=5)

folium.Choropleth(
    geo_data=merged_data,
    name="choropleth Map of Turkey Earthquake",
    data=merged_data,
    columns=["shapeISO", "magSum"],
    key_on="feature.properties.shapeISO",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Earthquake Magnitude sum per Province",
).add_to(choropMap)

folium.LayerControl().add_to(choropMap)

choropMap
```

In [ ]:
# Showcase of Reverse Geocoding
import reverse_geocoder as rg
g = rg.search(mapCtr)
print(g) # g is a list with one dictionary
print('City:', g[0]['name'], '| Province:', g[0]['admin1'], '| Country:', g[0]['cc'])


##### Function 3 - Reverse Geocoding - Takes 51 minutes, then comments out for this moment
```
import reverse_geocoder as rg
def get_location(row):
    coordinates = (row['latitude'], row['longitude'])
    result = rg.search(coordinates)[0]
    return result['name'], result['admin1'], result['cc']

df2[['city', 'state', 'country']] = df2.apply(get_location, axis=1, result_type='expand')

df2
```

## PART 7. Bootstrap the Landing page and Other pages

This has been done by bootstraping a very simple landing page, consisting of:
* a Toolbar with
    * Home
    * Data Table
    * Bubble Map
    * Heat Map
    * Project
* a Map container, where
    * the content shall be switched over from one to another once visitor clicks the links on the toolbar;
    * a default page shall be loaded one the website is open.

So far, all look good.

## The END